In [1]:
import os
import requests
import time
import pandas as pd
import openai
client = openai.OpenAI()

kaggle_data = pd.read_csv("data/kaggle_data.csv")

In [2]:
kaggle_data.columns

Index(['movie_name', 'imdb_id', 'character_name_kaggle', 'line_count', 'title',
       'director', 'genres', 'imdb_rating', 'year', 'plot', 'script_filename',
       'script_path', 'script_exists', 'Drama', 'Thriller', 'Comedy', 'Action',
       'Crime', 'Romance', 'Adventure', 'Sci-Fi', 'Mystery', 'Horror',
       'Fantasy', 'Biography', 'imdb_rating_decimal'],
      dtype='object')

In [8]:
if os.path.exists("data/intermediate_protagonists.csv"):
    protagonist_map = pd.read_csv("data/intermediate_protagonists.csv", index_col=0).to_dict()["0"]
    protagonist_map = {str(k).zfill(7): v for k, v in protagonist_map.items()}

else:
    protagonist_map = {}

In [9]:
protagonist_map

{'1179933': 'Michelle',
 '0147800': 'Kat Stratford',
 '0118528': 'Jack Lemmon ',
 '0114746': 'James Cole',
 '2024544': 'Solomon Northup',
 '1542344': 'Aron Ralston',
 '2991516': 'Jack ',
 '0450385': 'Mike Enslin',
 '0103594': 'Christopher Columbus ',
 '0179626': 'Eddie Flemming ',
 '0450232': 'Jack Mosley ',
 '0974661': "Mike O'Donnell",
 '8579674': 'Will Schofield',
 '1190080': 'Jackson Curtis',
 '4385888': 'Jamie Fields',
 '0315733': 'Paul Rivers ',
 '1232829': 'Morton Schmidt',
 '7280898': 'Viljar Hanssen ',
 '0307901': 'Montgomery Brogan',
 '0191754': 'Gwen Cummings',
 '0289043': 'Jim'}

In [11]:
for i in range(kaggle_data.shape[0]):
    imdb_id = kaggle_data.iloc[i]["imdb_id"]
    title = kaggle_data.iloc[i]["title"]
    if str(imdb_id).zfill(7) in protagonist_map:
        print(f"Skipping movie {title} as protagonist already found.")
        continue
    
    description = kaggle_data.iloc[i]["plot"]
    director = kaggle_data.iloc[i]["director"]

    prompt = f"""Search the internet for the protagonist of the following movie.
    Title: {title}
    Description: {description}
    Director: {director}
    Never write anything other than the name of the protagonist.
    No explanations, no full sentences, no references--just the name of the protagonist.
    If there is no clear protagonist, return an empty string.
    Protagonist name:"""

    response = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{"type": "web_search"}],
    input = prompt,
    temperature=0)
    answer = response.output_text

    #write raw output to protagonist map
    protagonist_map[str(imdb_id).zfill(7)] = answer
    if i % 10 == 0:
        print(i)
        pd.Series(protagonist_map).to_csv(f"data/intermediate_protagonists.csv")
pd.Series(protagonist_map).to_csv(f"data/intermediate_protagonists.csv")

Skipping movie 10 Cloverfield Lane as protagonist already found.
Skipping movie 10 Things I Hate About You as protagonist already found.
Skipping movie 12 Angry Men as protagonist already found.
Skipping movie 12 Monkeys as protagonist already found.
Skipping movie 12 Years a Slave as protagonist already found.
Skipping movie 127 Hours as protagonist already found.
Skipping movie 13/13/13 as protagonist already found.
Skipping movie 1408 as protagonist already found.
Skipping movie 1492: Conquest of Paradise as protagonist already found.
Skipping movie 15 Minutes as protagonist already found.
Skipping movie 16 Blocks as protagonist already found.
Skipping movie 17 Again as protagonist already found.
Skipping movie 1917 as protagonist already found.
Skipping movie 2012 as protagonist already found.
Skipping movie 20th Century Women as protagonist already found.
Skipping movie 21 Grams as protagonist already found.
Skipping movie 21 Jump Street as protagonist already found.
Skipping movi

In [12]:
pd.Series(protagonist_map).to_csv(f"data/intermediate_protagonists.csv")

In [17]:
#add protagonist from dict to kaggle_data in new column
kaggle_data["character_name_search"] = kaggle_data["imdb_id"].apply(lambda x: protagonist_map.get(str(x).zfill(7), ""))
kaggle_data.to_csv("data/kaggle_data.csv", index=False)